In [ ]:
import cv2
import numpy as np
from matplotlib import pyplot as plt

def call_track(pos):
    lux = cv2.getTrackbarPos('LU_x', 'dst')
    luy = cv2.getTrackbarPos('LU_y', 'dst')
    rux = cv2.getTrackbarPos('RU_x', 'dst')
    ruy = cv2.getTrackbarPos('RU_y', 'dst')
    ldx = cv2.getTrackbarPos('LD_x', 'dst')
    ldy = cv2.getTrackbarPos('LD_y', 'dst')
    rdx = cv2.getTrackbarPos('RD_x', 'dst')
    rdy = cv2.getTrackbarPos('RD_y', 'dst')
    
    pts1 = np.float32([[0,0], [0,rows], [cols, 0], [cols,rows]])
    pts2 = np.float32([[lux,luy], [ldx, ldy], [rux, ruy], [rdx, rdy]])
    mtrx = cv2.getPerspectiveTransform(pts1, pts2)
    dst = cv2.warpPerspective(img, mtrx, (cols, rows))
    dst1 = dst.copy()
    cv2.imshow('dst1', dst1)

img = cv2.imread('./Desktop/KakaoTalk_20220707_222728324.jpg')
rows, cols = img.shape[:2]

# center = (w/2, h/2)
# print('img2 shape = rows x cols = {} x {}'.format(rows, cols))
# img1 = img.copy()


cv2.circle(img, (0,0), 10, (255,0,0), -1)
cv2.circle(img, (0,rows), 10, (0,255,0), -1)
cv2.circle(img, (cols,0), 10, (0,0,255), -1)
cv2.circle(img, (cols,rows), 10, (0,255,255), -1)


# pts1 = np.float32([[0,0], [0,rows], [cols, 0], [cols,rows]])
# pts2 = np.float32([[100,50], [100,rows-50], [cols-100, 100], [cols-10,rows-50]])
# mtrx = cv2.getPerspectiveTransform(pts1, pts2)
# dst = cv2.warpPerspective(img, mtrx, (cols, rows))

    
cv2.namedWindow('dst')
# cv2.imshow('image', img)
# cv2.imshow('perspective', dst)
cv2.createTrackbar('LU_x', 'dst', 0, 480, call_track)
cv2.createTrackbar('LU_y', 'dst', 0, 640, call_track)
cv2.createTrackbar('LD_x', 'dst', 0, 480, call_track)
cv2.createTrackbar('LD_y', 'dst', 480, 480, call_track)
cv2.createTrackbar('RU_x', 'dst', 640, 640, call_track)
cv2.createTrackbar('RU_y', 'dst', 0, 480, call_track)
cv2.createTrackbar('RD_x', 'dst', 640, 640, call_track)
cv2.createTrackbar('RD_y', 'dst', 480, 480, call_track)

cv2.waitKey()
cv2.destroyAllWindows()

In [6]:
# 어핀 변환 (getAffine.py)

import cv2
import numpy as np
from matplotlib import pyplot as plt

file_name = './Desktop/KakaoTalk_20220707_222728324.jpg'
img = cv2.imread(file_name)
rows, cols = img.shape[:2]


#---① 원근 변환 전 후 4개 좌표
pts1 = np.float32([[0,0], [0,rows], [cols, 0], [cols,rows]])
pts2 = np.float32([[50,50], [0,rows], [cols, 0], [cols,rows]])

# ---② 변환 전 좌표를 원본 이미지에 표시
cv2.circle(img, (100,100), 10, (255,0,0), -1)
cv2.circle(img, (0,rows), 10, (0,255,0), -1)
cv2.circle(img, (cols,0), 10, (0,0,255), -1)
cv2.circle(img, (cols,rows), 10, (0,255,255), -1)

#---③ 원근 변환 행렬 계산
mtrx = cv2.getPerspectiveTransform(pts1, pts2)
#---④ 원근 변환 적용
dst = cv2.warpPerspective(img, mtrx, (cols, rows))

cv2.imshow("origin", img)
cv2.imshow('perspective', dst)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [1]:
import cv2
import numpy as np

win_name = "scanning"
img = cv2.imread("./Desktop/istockphoto-462601397-612x612.jpg")
rows, cols = img.shape[:2]
draw = img.copy()
pts_cnt = 0
pts = np.zeros((4, 2), dtype=np.float32)

def onMouse(event, x, y, flags, param):
    global pts_cnt
    if event == cv2.EVENT_LBUTTONDOWN:
        # 좌표에 초록색 동그라미 표시
        cv2.circle(draw, (x, y), 10, (0, 255, 0), -1)
        cv2.imshow(win_name, draw)

        # 마우스 좌표 저장
        pts[pts_cnt] = [x, y]
        pts_cnt += 1
        if pts_cnt == 4:
            # 좌표 4개 중 상하좌우 찾기
            sm = pts.sum(axis=1)  # 4쌍의 좌표 각각 x+y 계산
            diff = np.diff(pts, axis=1)  # 4쌍의 좌표 각각 x-y 계산

            topLeft = pts[np.argmin(sm)]  # x+y가 가장 값이 좌상단 좌표
            bottomRight = pts[np.argmax(sm)]  # x+y가 가장 큰 값이 우하단 좌표
            topRight = pts[np.argmin(diff)]  # x-y가 가장 작은 것이 우상단 좌표
            bottomLeft = pts[np.argmax(diff)]  # x-y가 가장 큰 값이 좌하단 좌표

            # 변환 전 4개 좌표 
            pts1 = np.float32([topLeft, topRight, bottomRight, bottomLeft])

            # 변환 후 영상에 사용할 서류의 폭과 높이 계산
            w1 = abs(bottomRight[0] - bottomLeft[0])
            w2 = abs(topRight[0] - topLeft[0])
            h1 = abs(topRight[1] - bottomRight[1])
            h2 = abs(topLeft[1] - bottomLeft[1])
            width = max([w1, w2])  # 두 좌우 거리간의 최대값이 서류의 폭
            height = max([h1, h2])  # 두 상하 거리간의 최대값이 서류의 높이

            # 변환 후 4개 좌표
            pts2 = np.float32([[0, 0], [width - 1, 0],
                               [width - 1, height - 1], [0, height - 1]])

            # 변환 행렬 계산 
            mtrx = cv2.getPerspectiveTransform(pts1, pts2)
            # 원근 변환 적용
            result = cv2.warpPerspective(img, mtrx, (width, height))
            cv2.imshow('scanned', result)

cv2.imshow(win_name, img)
cv2.setMouseCallback(win_name, onMouse)
cv2.waitKey(0)
cv2.destroyAllWindows()

error: OpenCV(4.6.0) :-1: error: (-5:Bad argument) in function 'warpPerspective'
> Overload resolution failed:
>  - Can't parse 'dsize'. Sequence item with index 0 has a wrong type
>  - Can't parse 'dsize'. Sequence item with index 0 has a wrong type
